In [5]:
import json
from typing import Dict

import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import numpy as np
from lonboard import Map, ScatterplotLayer

In [13]:
BASE_URL = "http://127.0.0.1:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"
AGGREGATION_ENDPOINT = f"{BASE_URL}/aggregate"

In [7]:
def fetch_admin_boundaries(iso3: str, adm: str) -> gpd.GeoDataFrame:
    """Fetch administrative boundaries from GeoBoundaries API."""
    url = f'https://www.geoboundaries.org/api/current/gbOpen/{iso3}/{adm}/'
    res = requests.get(url).json()
    return gpd.read_file(res['gjDownloadURL'])

def fetch_summary_data(feature: Dict) -> pd.DataFrame:
    """Fetch summary data for each administrative feature."""
    request_payload = {
        "aoi": feature,
        "spatial_join_method": "touches",
        "fields": ["sum_pop_2020"], 
        "geometry": "point"
    }
    response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
    if response.status_code != 200:
        raise Exception(f"Failed to get summary: {response.text}")
    
    summary_data = response.json()
    if not summary_data:
        print(f"Failed to get summary for {feature['id']}")
        return pd.DataFrame()  # Return an empty DataFrame if no data
    
    df = pd.DataFrame(summary_data)
    df['adm_id'] = int(feature['id'])
    df['adm_name'] = feature['properties']['shapeName']
    df['geometry'] = df['geometry'].apply(lambda geom: shape(geom))
    return df

In [9]:
ISO3 = 'KEN'
ADM = 'ADM1'
adm_boundaries = fetch_admin_boundaries(ISO3, ADM)
geojson_str = adm_boundaries.to_json()
adm_geojson = json.loads(geojson_str)
adm_features = adm_geojson['features']

gdfs = []
for i, feature in enumerate(adm_features):
    df = fetch_summary_data(feature)
    if not df.empty:
        gdfs.append(gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326'))
        print(feature['properties']['shapeName'], i)

# Concatenate all GeoDataFrames into a single GeoDataFrame
gdf = pd.concat(gdfs, ignore_index=True)

# Display the GeoDataFrame structure
gdf.head()

Turkana 0
Marsabit 1
Mandera 2
Wajir 3
West Pokot 4
Samburu 5
Isiolo 6
Baringo 7
Elgeyo-Marakwet 8
Trans Nzoia 9
Bungoma 10
Garissa 11
Uasin Gishu 12
Kakamega 13
Laikipia 14
Busia 15
Meru 16
Nandi 17
Siaya 18
Nakuru 19
Vihiga 20
Nyandarua 21
Tharaka 22
Kericho 23
Kisumu 24
Nyeri 25
Tana River 26
Kitui 27
Kirinyaga 28
Embu 29
Homa Bay 30
Bomet 31
Nyamira 32
Narok 33
Kisii 34
Murang'a 35
Migori 36
Kiambu 37
Machakos 38
Kajiado 39
Nairobi 40
Makueni 41
Lamu 42
Kilifi 43
Taita Taveta 44
Kwale 45
Mombasa 46


hex_id                  geometry  sum_pop_2020  adm_id adm_name
0  866a4a407ffffff  POINT (35.80748 4.26557)    554.655021       0  Turkana
1  866a4a417ffffff   POINT (34.74751 4.3078)    207.977046       0  Turkana
2  866a4a41fffffff  POINT (34.20173 4.41427)    637.218817       0  Turkana
3  866a4a427ffffff  POINT (34.94623 4.06953)    948.876730       0  Turkana
4  866a4a437ffffff  POINT (35.51141 3.84781)    112.805780       0  Turkana

In [10]:
# Define custom breaks and corresponding RGBA colors for visualization
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

def assign_color(value: float) -> list:
    """Assign colors based on population value."""
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i].tolist()  # Convert to list
    return colors[-1].tolist()  # In case value exceeds all breaks

In [11]:
# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(assign_color)

# Flatten the color list into a 2D array for ScatterplotLayer
color_list = np.array(gdf['color'].tolist(), dtype=np.uint8)

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=color_list)

# Create the map with the scatterplot layer
m = Map(layer)
m

Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x14ec061a0>
[
  [
    2…

In [12]:
adm_features

[{'id': '0',
  'type': 'Feature',
  'properties': {'shapeName': 'Turkana',
   'shapeISO': 'KE-43',
   'shapeID': '32016919B72266624462344',
   'shapeGroup': 'KEN',
   'shapeType': 'ADM1'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[36.05060958836185, 4.456217766236989],
     [35.94394683797623, 4.548029422619834],
     [35.93885421734831, 4.584362507224],
     [35.949348448572096, 4.628556251969485],
     [35.81214523321597, 4.782410621852762],
     [35.81184768712109, 5.092334270410277],
     [35.81184387219702, 5.095732688812575],
     [35.821193695432726, 5.106872558621717],
     [35.82409668001782, 5.109823227060303],
     [35.825977325193435, 5.111377239368153],
     [35.82680892918876, 5.112345695897204],
     [35.82710647618296, 5.113322734656549],
     [35.827430724948044, 5.115200996628744],
     [35.82780837995284, 5.116295337559848],
     [35.82852172759573, 5.117104529550431],
     [35.830711365437196, 5.118960380417263],
     [35.83223724335414, 5.1204833975907

In [15]:
def fetch_aggregated_population(row):
    request_payload = {
        "aoi": {
            "type": "Feature",
            "geometry": row.geometry.__geo_interface__,
            "properties": {}
        },
        "spatial_join_method": "touches",
        "fields": ["sum_pop_2020"],
        "aggregation_type": "sum"
    }

    response = requests.post(AGGREGATION_ENDPOINT, json=request_payload)

    if response.status_code == 200:
        result = response.json()
        return result.get("sum_pop_2020", 0)
    else:
        return 0


In [18]:
gdf = gpd.GeoDataFrame.from_features(adm_features)
gdf['sum_pop_2020'] = gdf.apply(fetch_aggregated_population, axis=1)
gdf.head()

geometry        shapeName  \
0   POLYGON ((36.05061 4.45622, 35.94395 4.54803, ...          Turkana   
1   POLYGON ((36.60089 2.40574, 36.60138 2.4053, 3...         Marsabit   
2   POLYGON ((40.99195 2.17919, 40.99245 2.25188, ...          Mandera   
3   POLYGON ((38.96255 2.09739, 38.96272 2.09718, ...            Wajir   
4   POLYGON ((34.94278 2.45547, 34.93892 2.45551, ...       West Pokot   
5   POLYGON ((36.39206 0.91877, 36.39192 0.91835, ...          Samburu   
6   POLYGON ((37.94529 1.26288, 37.94816 1.19432, ...           Isiolo   
7   POLYGON ((35.79194 1.66362, 35.79185 1.66355, ...          Baringo   
8   POLYGON ((35.15214 1.19937, 35.16018 1.18046, ...  Elgeyo-Marakwet   
9   POLYGON ((34.82229 1.26259, 34.82192 1.26249, ...      Trans Nzoia   
10  POLYGON ((34.58077 1.15033, 34.58031 1.15027, ...          Bungoma   
11  POLYGON ((39.46069 0.99468, 39.4606 0.99454, 3...          Garissa   
12  POLYGON ((35.58744 0.1695, 35.58558 0.16949, 3...      Uasin Gishu   
13  POLYGON ((35.01771 0.88754, 35.01777 0.88714, ...         Kakamega   
14  POLYGON ((36.48872 0.8468, 36.47331 0.8385, 36...         Laikipia   
15  POLYGON ((34.36053 0.77841, 34.36019 0.77841, ...            Busia   
16  POLYGON ((37.38583 0.27442, 37.38543 0.2741, 3...             Meru   
17  POLYGON ((34.8503 0.54573, 34.87022 0.53017, 3...            Nandi   
18  POLYGON ((34.37066 0.30656, 34.37043 0.30635, ...            Siaya   
19  POLYGON ((35.64015 -0.06097, 35.63872 -0.06193...           Nakuru   
20  POLYGON ((34.56035 0.09393, 34.56017 0.09377, ...           Vihiga   
21  POLYGON ((36.25549 -0.02196, 36.25241 -0.03496...        Nyandarua   
22  POLYGON ((37.30779 -0.15107, 37.30811 -0.15135...          Tharaka   
23  POLYGON ((35.5228 0.01541, 35.51604 0.01586, 3...          Kericho   
24  POLYGON ((34.74707 -0.02473, 34.7458 -0.02473,...           Kisumu   
25  POLYGON ((36.65747 -0.15308, 36.68586 -0.17446...            Nyeri   
26  POLYGON ((38.43415 -0.07482, 38.82981 -0.76018...       Tana River   
27  POLYGON ((38.4207 -0.06928, 38.42054 -0.06922,...            Kitui   
28  POLYGON ((37.16912 -0.63562, 37.16915 -0.63586...        Kirinyaga   
29  POLYGON ((37.30779 -0.15107, 37.41457 -0.39432...             Embu   
30  POLYGON ((33.94766 -0.33718, 33.9281 -0.45482,...         Homa Bay   
31  POLYGON ((35.45109 -0.3654, 35.45009 -0.36632,...            Bomet   
32  POLYGON ((35.01961 -0.4124, 35.01892 -0.41273,...          Nyamira   
33  POLYGON ((35.58665 -0.58811, 35.39608 -0.64488...            Narok   
34  POLYGON ((34.63766 -0.66513, 34.63659 -0.67013...            Kisii   
35  POLYGON ((36.72004 -0.80637, 36.72087 -0.80709...         Murang'a   
36  POLYGON ((33.92824 -0.80242, 33.93507 -1.0011,...           Migori   
37  POLYGON ((36.53373 -1.15591, 36.51061 -1.24019...           Kiambu   
38  POLYGON ((37.84551 -0.80983, 37.84511 -0.80993...         Machakos   
39  POLYGON ((36.33392 -1.04937, 36.34618 -1.17568...          Kajiado   
40  POLYGON ((36.66389 -1.31521, 36.65987 -1.32207...          Nairobi   
41  POLYGON ((37.75332 -1.56638, 37.75275 -1.56655...          Makueni   
42  MULTIPOLYGON (((40.20794 -2.03289, 40.21904 -2...             Lamu   
43  MULTIPOLYGON (((39.21737 -3.07293, 39.15588 -3...           Kilifi   
44  POLYGON ((39.07511 -3.04114, 39.07503 -3.04111...     Taita Taveta   
45  MULTIPOLYGON (((39.09229 -3.56708, 39.06294 -3...            Kwale   
46  MULTIPOLYGON (((39.64795 -3.95101, 39.64832 -3...          Mombasa   

   shapeISO                  shapeID shapeGroup shapeType  sum_pop_2020  
0     KE-43  32016919B72266624462344        KEN      ADM1  1.354744e+06  
1     KE-25  32016919B63496705134089        KEN      ADM1  4.205930e+05  
2     KE-24   32016919B2031803566233        KEN      ADM1  2.739353e+06  
3     KE-46  32016919B89873713911655        KEN      ADM1  1.901271e+06  
4     KE-47  32016919B96045830258165        KEN      ADM1  9.151929e+05  
5     KE-37  32016919B77230315601578        KEN 

In [41]:
breaks = [0, 10_000, 100_000, 1_000_000, 5_000_000, 10_000_000, 20_000_000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 125],  # Light gray with transparency for 0 - 10,000
    [255, 255, 0, 125],    # Yellow with transparency for 10,000 - 100,000
    [255, 165, 0, 125],    # Orange with transparency for 100,000 - 1,000,000
    [255, 0, 0, 125],      # Red with transparency for 1,000,000 - 5,000,000
    [128, 0, 128, 125],    # Purple with transparency for 5,000,000 - 10,000,000
    [0, 0, 255, 125],      # Blue with transparency for 10,000,000 - 20,000,000
    [0, 0, 139, 125],      # Dark blue with transparency for 20,000,000 - max
])

def assign_color(value: float) -> list:
    """Assign colors based on population value, including transparency."""
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i].tolist()  # Convert to list
    return colors[-1].tolist()

In [42]:
from lonboard import PolygonLayer
import numpy as np

# Assign colors to each row in gdf based on sum_pop_2020
gdf['color'] = gdf["sum_pop_2020"].apply(assign_color)

# Convert the list of colors into a 2D numpy array of uint8
color_array = np.array(gdf['color'].tolist(), dtype=np.uint8)

# Create the polygon layer using PolygonLayer, referencing the color_array
layer = PolygonLayer.from_geopandas(
    gdf,
    get_fill_color=color_array,  # Pass the 2D numpy array for the colors
    get_line_color=[0, 0, 0, 255],  # Optional: Black outline
    line_width_min_pixels=1
)

# Create the map with the polygon layer
m = Map(layer)
m

/opt/anaconda3/envs/spacestats/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(layers=[PolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x15bb0bc40>
[
  [
    255,
…